<a href="https://colab.research.google.com/github/EmilGauti/flog/blob/master/flog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import numpy as np
import features as features

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#input: 
#filename: name of file to read from
#shape: dimension of resulting array                   
def read3DArrayFromFile(fileName,shape):                      
    data = np.loadtxt(fileName, dtype = 'float32')
    data = data.reshape(shape)
    return(data)

nrPatients = 24
fs = 256 # Sampling rate
nrChannels = 23
seizureLength = 14 # seconds
nrSeizures = 170
fRange = 129
M = 16
fLowerLimit = 0.5
fUpperLimit = 25
dataShape = (nrSeizures,nrChannels,seizureLength*fs)

# Read raw data
seizureData = read3DArrayFromFile('seizureChunks14.txt',dataShape)
nonSeizureData = read3DArrayFromFile('nonSeizureChunks14.txt',dataShape)
allData = np.concatenate((seizureData,nonSeizureData),axis = 0)

# Calculate EEG features
[X,y] = create_data_matrix(allData, True, nrSeizures, nrChannels, fRange, fs, M, fLowerLimit, fUpperLimit)
print("X:", X.shape)
print("y:", y.shape)

print('Logistic Regression classifier trained on data from pooled subjects')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
clf=LogisticRegression(max_iter  = 200)
clf.fit(X_train, y_train)
print("Accuracy: ", clf.score(X_test, y_test))


X: (340, 437)
y: (340,)
Logistic Regression classifier trained on data from pooled subjects


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy:  0.7176470588235294


In [0]:
print(allData.shape)

(340, 23, 3584)


In [0]:
# FEATURES

import numpy as np
from scipy import signal

# Hjorth parameters (time domain)
# https://en.wikipedia.org/wiki/Hjorth_parameters
def hjorth_mobility(x):
    num = np.var(np.diff(x))
    den = np.var(x)
    if den > 0:
        return np.sqrt(num / den)
    else:
        return 0.0

def hjorth_parameters(x):
    activity=np.var(x)
    mobility=hjorth_mobility(x)
    if mobility > 0:
        complexity=hjorth_mobility(np.diff(x)) / mobility
    else:
        complexity=0.0
    return np.array([activity, mobility, complexity])

#Calculates power spectral density for an eeg segment
#inputs: 
#signalMat: signal matrix for chunk
#fs: sampling density
#n: number of channels
#outputs: 
#f: frequency
#Pwelch: power spectral density calculated by Welch's method
def psd(signalMat,fs,n,fRange):
    Pwelch = np.zeros((n,fRange))
    for i in range(n):
        F,Pwelch[i,:] = signal.welch(signalMat[i,:],fs,scaling = 'spectrum')
    return(F,Pwelch)

# Absolute band power
# Combined power in M frequency bands
def absolute_power(f, PSD,M,l,h):
    length = (h-l)/M
    power = []
    k = l
    for i in range(M):
        power.append(sum(PSD[np.where((f > k) & (f <= k+length))]))
        k +=length
    return(power)
    
#Relative power of delta, theta, alpha 
#and beta waves for a single channel
def relative_power(f,PSD,M,l,h):
    absPow = absolute_power(f,PSD,M,l,h)
    tot = sum(PSD)
    if tot > 0.0:
        return(absPow/tot)
    else:
        return 0.0

# Calculate relative band power for the whole data set
# THINK: Might want to do the same for absolute power
def relative_power_all(allData, nrSeizures, nrChannels, fRange, fs, M, fLowerLimit, fUpperLimit):
    dataPSD = np.zeros((nrSeizures*2,nrChannels,fRange))
    for i in range(nrSeizures*2):
        [F,dataPSD[i,:,:]] = psd(allData[i,:,:],fs,nrChannels,fRange)

    dataRelPower = np.zeros((nrSeizures*2,nrChannels,M))
    for i in range(nrSeizures*2):
        for j in range(nrChannels):
            dataRelPower[i,j,:] = relative_power(F,dataPSD[i,j,:],M,fLowerLimit,fUpperLimit)
    return(dataRelPower)
    
def create_data_matrix(allData, flatten, nrSeizures, nrChannels, fRange, fs, M, fLowerLimit, fUpperLimit):
    dataRelPower = relative_power_all(allData, nrSeizures, nrChannels, fRange, fs, M, fLowerLimit, fUpperLimit)

    n = allData.shape[0]
    hjopar = np.zeros((n,3*23))
    for i in range(n):
      for j in range(23):
        hjopar[i,3*j:(3*j+3)] = hjorth_parameters(allData[i,j,:])
        
    if flatten:
        dataRelPowerFlat = np.zeros((nrSeizures*2,M*nrChannels))
        for i in range(nrSeizures*2):
            dataRelPowerFlat[i,:] = dataRelPower[i,:,:].flatten()
        dataRelPowerFlat = np.c_[dataRelPowerFlat, hjopar]
        X = dataRelPowerFlat
        y = np.concatenate((np.repeat(1,nrSeizures),np.repeat(0,nrSeizures)))
    else:
        # (channels*features matrix, e.g. for use in convolutional nets)
        X = np.expand_dims(dataRelPower,axis = 3)
        y = np.concatenate((np.repeat(1,nrSeizures),np.repeat(0,nrSeizures)))
    return(X,y)


In [0]:
A = np.array([[1,2,3], [4,5,6]])
B = np.array([[10], [10]])

print(np.c_[A, B])

[[ 1  2  3 10]
 [ 4  5  6 10]]


In [0]:
# PATIENT SPECIFIC

# -*- coding: utf-8 -*-

# Patient-specific classifier

# Pre: Execute main.py

import ast

def fixIndex(i):
    if i < 10:
        i = '0'+str(i)
    return(str(i))

nonSeizureFileNames = open('nonSeizureFileNames.txt', 'r')
nonSeizureFileNames = nonSeizureFileNames.read().split('\n')

with open('seizureDict.txt', 'r') as f:
    s = f.read()
    seizureDict = ast.literal_eval(s)

scores = np.zeros((nrPatients))

for testPatient in range(1,nrPatients+1):
    #gets index of seizures that belong to testPatient (+nrSeizures since seizure chunks are first in allData)
    prefix = 'chb'+fixIndex(testPatient)
    nonSeizIndices = [i+nrSeizures for i, s in enumerate(nonSeizureFileNames) if s.startswith(prefix)]
    testPatientKeys = [i for i in seizureDict.keys() if i.startswith(prefix)]
    k = 0
    seizIndices = []
    for key in seizureDict:    
        for i in range(len(seizureDict[key])):
            if key in testPatientKeys:
                seizIndices.append(k)
            k = k+1

    print("Patient ", testPatient, "n_seizures=",len(seizIndices), "n_nonseizures=",len(nonSeizIndices))
    
    X_test = np.concatenate((X[seizIndices,:],X[nonSeizIndices,:]),axis = 0)
    y_test = np.concatenate((y[seizIndices],y[nonSeizIndices]))
    X_train = np.delete(X, nonSeizIndices, axis=0)
    X_train = np.delete(X_train, seizIndices, axis=0)
    y_train = np.delete(y,nonSeizIndices)
    y_train = np.delete(y_train,seizIndices)
    # THINK: Collect statistics on seizure/nonseizure
    print("Train:", X_train.shape, "Test:", X_test.shape)

    # Classify individual patient
    # Insert code here ...
    clf=LogisticRegression(max_iter = 200)
    clf.fit(X_train, y_train)
    scores[testPatient-1] = clf.score(X_test, y_test)
    print("Accuracy: ", scores[testPatient-1])

print('Mean accuracy:', np.mean(scores))
#print('Accuracy Quantiles:', np.quantile(scores, q = [0, 0.25, 0.5, 0.75, 1]))

Patient  1 n_seizures= 7 n_nonseizures= 11
Train: (322, 437) Test: (18, 437)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy:  0.9444444444444444
Patient  2 n_seizures= 2 n_nonseizures= 15
Train: (323, 437) Test: (17, 437)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy:  0.7647058823529411
Patient  3 n_seizures= 7 n_nonseizures= 5
Train: (328, 437) Test: (12, 437)


...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy:  0.875
Patient  24 n_seizures= 16 n_nonseizures= 3
Train: (321, 437) Test: (19, 437)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy:  0.8421052631578947
Mean accuracy: 0.6733574084466362


AttributeError: ignored

In [0]:
print('Mean accuracy:', np.quantile(scores, 0.5))

AttributeError: ignored